#  Chunker de Textos - Implantação

* Este componente requer conexão com a internet para o download online de dependências do NLTK.
* Este componente faz janelamento de textos longos, dividindo um texto em partes menores, cada parte de texto criada (chunk), possui uma quantidade de elementos (palavras ou sentenças) pré-definidas pelo usuário; como também a quantidade de elementos que se sobrepõem entre cada chunk consecutivo. 
* Exemplo:
        
    - Texto: "Hoje o dia amanheceu ensolarado, vou fazer uma caminhada e ouvir os sons da natureza."

    - Hiperparâmetros: {
        "chunkenizer": "word",
        "chunk_size": 5,
        "chunk_overlap": 2
    }

    - Saída: [
        "Hoje o dia amanheceu ensolarado,",
        "amanheceu ensolarado, vou fazer uma",
        "fazer uma caminhada e ouvir",
        "e ouvir os sons da",
        "sons da natureza."
    ]


### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [2]:
%%writefile Model.py

import joblib
import numpy as np
import pandas as pd
from chunker import Chunker
from utils import generate_dataframe

class Model:
    def __init__(self):
        self.loaded = False

    def load(self):
        
        # Load artifacts
        artifacts = joblib.load("/tmp/data/chunker.joblib")
        self.model_parameters = artifacts["model_parameters"]
        self.inference_parameters = artifacts["inference_parameters"]
        
        # Initialize chunker
        self.chunker = Chunker(**self.model_parameters)
        
        # Set model loaded
        self.loaded = True
        print("Loaded model")
    
    def class_names(self):
        column_names = list(self.inference_parameters['columns']) + [self.inference_parameters['output_column_name']]
        return column_names

    def predict(self, X, feature_names, meta=None):
        if not self.loaded:
            self.load()
            
        if feature_names:
            # Antes de utilizar o conjunto de dados X no modelo, reordena suas features de acordo com a ordem utilizada no treinamento
            df = pd.DataFrame(X, columns=feature_names)
            X = df[self.inference_parameters['columns']]
        
        else:
            X = pd.DataFrame(X, columns=self.inference_parameters['columns'])
            
        # Generate Chunks
        chunks = self.chunker(X[self.inference_parameters['text_column_name']])
        
        # Generate Dataframe
        if self.inference_parameters['replicate_data'] == 'sim':
            # Replicate Data
            output_df = generate_dataframe(X, chunks, self.inference_parameters)
        else:
            output_df = X.copy()
            output_df[self.inference_parameters['output_column_name']] = chunks
        
        # Output
        output = output_df.to_numpy()

        return output

Overwriting Model.py
